<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective<a class="jp-InternalAnchorLink" href="#Objective" target="_self">&para;</a></h3>
<br><strong style="color: #000000;"><em>Check and Rotate Expiring Access Keys for all IAM Users </em></strong></div>
</center><center>
<h2 id="AWS-Access-Key-Rotation"><u>AWS Access Key Rotation</u><a class="jp-InternalAnchorLink" href="#AWS-Access-Key-Rotation" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h1>
<p>1) <a href="#2" target="_self" rel="noopener">List all Expiring Access Key</a><br>2)&nbsp;<a href="#3" target="_self" rel="noopener">Create AWS Access Key</a><br>3) <a href="#4" target="_self" rel="noopener">Update AWS Access Key</a><br>4)&nbsp;<a href="#5" target="_self" rel="noopener">Delete AWS Access Key</a></p>

<h3 id="List-all-Expiring-Access-Keys"><a id="2" target="_self" rel="nofollow"></a>List all Expiring Access Keys<a class="jp-InternalAnchorLink" href="#List-all-Expiring-Access-Keys" target="_self">&para;</a></h3>
<p>Using unSkript's AWS List Expiring Access Keys action we will list those users whose Access Keys past the given threshold number of days i.e. expiring.</p>
<blockquote>
<p>Action takes the following parameters: <code>threshold_days</code></p>
</blockquote>
<blockquote>
<p>Action captures the following output: <code>expiring_users</code></p>
</blockquote>

In [ ]:
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import pprint
from typing import Tuple
import datetime
import dateutil
from pydantic import BaseModel, Field
from unskript.legos.aws.aws_list_all_iam_users.aws_list_all_iam_users import aws_list_all_iam_users

from beartype import beartype
@beartype
def aws_list_expiring_access_keys_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_list_expiring_access_keys(handle, threshold_days: int = 90)-> Tuple:
    """aws_list_expiring_access_keys returns all the ACM issued certificates which are
       about to expire given a threshold number of days

        :type handle: object
        :param handle: Object returned from Task Validate

        :type threshold_days: int
        :param threshold_days: Threshold number of days to check for expiry. Eg: 30 -lists
        all access Keys which are expiring within 30 days

        :rtype: Status, List of expiring access keys and Error if any
    """
    result =[]
    all_users=[]
    try:
        all_users = aws_list_all_iam_users(handle=handle)
    except Exception as error:
        raise error

    for each_user in all_users:
        try:
            iamClient = handle.client('iam')
            final_result={}
            response = iamClient.list_access_keys(UserName=each_user)
            for x in response["AccessKeyMetadata"]:
                if len(response["AccessKeyMetadata"])!= 0:
                    create_date = x["CreateDate"]
                    right_now = datetime.datetime.now(dateutil.tz.tzlocal())
                    diff = right_now-create_date
                    days_remaining = diff.days
                    if days_remaining > threshold_days:
                        final_result["username"] = x["UserName"]
                        final_result["access_key_id"] = x["AccessKeyId"]
            if len(final_result)!=0:
                result.append(final_result)
        except Exception as e:
            raise e

    if len(result) != 0:
        return (False, result)
    return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "threshold_days": "int(threshold_days)"
    }''')
task.configure(outputName="expiring_users")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_expiring_access_keys, lego_printer=aws_list_expiring_access_keys_printer, hdl=hdl, args=args)

<h3 id="List-of-Expiring-Users">List of Expiring Users</h3>
<p>This action lists the usernames of expiring Access Keys using the output from Step 2.</p>

In [21]:
all_expiring_users = []
if expiring_users[0] == False:
    if len(expiring_users[1])!=0:
        all_expiring_users=expiring_users[1]
print(all_expiring_users)

<h3 id="List-of-Expiring-Users-and-Access-Keys"><a id="3" target="_self" rel="nofollow"></a>List of Expiring Users and Access Keys</h3>
<p>This action simply creates another list containing a dictionary of the user and their old access key. The output from this acion is required for Step 4 and Step 5.</p>

In [22]:
expiring_usernames = []
for each_user in all_expiring_users:
    for k,v in each_user.items():
        if k=='username':
            expiring_usernames.append(v)
print(expiring_usernames)
task.configure(outputName="expiring_usernames")

<h3 id="Create-AWS-Access-Keys"><a id="3" target="_self" rel="nofollow"></a>Create AWS Access Keys</h3>
<p>Using unSkript's AWS Create Access Key action we will create a new Access Key for the users from Step 2.</p>
<blockquote>
<p>Action takes the following parameters: <code>aws_username</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import pprint
from typing import Dict
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def aws_create_access_key_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_create_access_key(
    handle,
    aws_username: str
) -> Dict:
    """aws_create_access_key creates a new access key for the given user.
        :type handle: object
        :param handle: Object returned from Task Validate

        :type aws_username: str
        :param aws_username: Username of the IAM user to be looked up

        :rtype: Result Dictionary of result
    """
    iamClient = handle.client('iam')
    result = iamClient.create_access_key(UserName=aws_username)
    retVal = {}
    temp_list = []
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "aws_username": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "expiring_usernames",
    "iter_parameter": "aws_username"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(expiring_usernames)!=0",
    "condition_result": true
    }''')

task.configure(outputName="new_access_keys")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_access_key, lego_printer=aws_create_access_key_printer, hdl=hdl, args=args)

<h3 id="Update-AWS-Access-Key"><a id="4" target="_self" rel="nofollow"></a>Update AWS Access Key</h3>
<p>Using the AWS Update Access Key action we will update the status of the old Access Key to <strong>"Inactive"</strong>. This step is required to delete the old access key as one user cannot have two Access Keys.</p>
<blockquote>
<p>This action takes the following parameters: <code>aws_username</code>, <code>aws_access_key_id</code> and <code>status</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import pprint
from typing import Dict
from pydantic import BaseModel, Field
from unskript.enums.aws_access_key_enums import AccessKeyStatus


from beartype import beartype
@beartype
def aws_update_access_key_printer(output):
    if output is None:
        return
    pprint.pprint("Access Key status successfully changed")
    pprint.pprint(output)


@beartype
def aws_update_access_key(
    handle,
    aws_username: str,
    aws_access_key_id: str,
    status: AccessKeyStatus
) -> Dict:
    """aws_update_access_key updates the status of an access key to Inactive/Active
        :type handle: object
        :param handle: Object returned from Task Validate

        :type aws_username: str
        :param aws_username: Username of the IAM user to be looked up

        :type aws_access_key_id: str
        :param aws_access_key_id: Old Access Key ID of the user of which the status
        needs to be updated

        :type status: AccessKeyStatus
        :param status: Status to set for the Access Key

        :rtype: Result Dictionary of result
    """
    iamClient = handle.client('iam')
    result = iamClient.update_access_key(
        UserName=aws_username,
        AccessKeyId=aws_access_key_id,
        Status=status
        )
    retVal = {}
    temp_list = []
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(inputParamsJson='''{
    "aws_username": "\\"iter.get(\\\\\\"username\\\\\\")\\"",
    "aws_access_key_id": "\\"iter.get(\\\\\\"access_key_id\\\\\\")\\"",
    "status": "AccessKeyStatus.Inactive"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_expiring_users",
    "iter_parameter": ["aws_access_key_id","aws_username"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_expiring_users)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_update_access_key, lego_printer=aws_update_access_key_printer, hdl=hdl, args=args)

<h3 id="Delete-AWS-Access-Key"><a id="5" target="_self" rel="nofollow"></a>Delete AWS Access Key</h3>
<p>Finally, we will delete the the old (Inactive) Access Key for the IAM Users</p>
<blockquote>
<p>This action takes the following parameters: <code>aws_username</code> and <code>aws_access_key_id</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import pprint
from typing import Dict
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def aws_delete_access_key_printer(output):
    if output is None:
        return
    pprint.pprint("Access Key successfully deleted")
    pprint.pprint(output)


@beartype
def aws_delete_access_key(
    handle,
    aws_username: str,
    aws_access_key_id: str,
) -> Dict:
    """aws_delete_access_key deleted the given access key.
        :type handle: object
        :param handle: Object returned from Task Validate

        :type aws_username: str
        :param aws_username: Username of the IAM user to be looked up

        :type aws_access_key_id: str
        :param aws_access_key_id: Old Access Key ID of the user which needs to be deleted

        :rtype: Result Status Dictionary of result
    """
    iamClient = handle.client('iam')
    result = iamClient.delete_access_key(UserName=aws_username, AccessKeyId=aws_access_key_id)
    retVal = {}
    temp_list = []
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "aws_username": "iter.get(\\"username\\")",
    "aws_access_key_id": "iter.get(\\"access_key_id\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_expiring_users",
    "iter_parameter": ["aws_username","aws_access_key_id"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_expiring_users)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_access_key, lego_printer=aws_delete_access_key_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion<a class="jp-InternalAnchorLink" href="#Conclusion" target="_self">&para;</a></h3>
<p>In this Runbook, we were able to perform AWS Access Key rotation for IAM users whose Access Keys were expiring by using unSkript's AWS actions. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>